In [1]:
import os
import sys
sys.path.append('/'.join(os.getcwd().split('/')[:-1]))

In [2]:
from model.bAbI_utils_loader import bAbIDataset, bAbIDataLoader

## new_version

Train

In [3]:
path = '../data/QA_bAbI_tasks/en-10k/qa1_single-supporting-fact_train.txt'

In [4]:
bAbI = bAbIDataset(path, train=True, return_masks=False)
train_loader = bAbIDataLoader(dataset=bAbI, batch_size=32, shuffle=True, to_tensor=True)

In [5]:
for batch in train_loader.load():
    stories, questions, answers, supports = batch
    break

In [6]:
print(stories.size(), questions.size(), answers.size())

torch.Size([32, 10, 6]) torch.Size([32, 4]) torch.Size([32, 1])


In [7]:
bAbI = bAbIDataset(path, train=True, return_masks=True)
train_loader = bAbIDataLoader(dataset=bAbI, batch_size=32, shuffle=True, to_tensor=True)

In [8]:
for batch in train_loader.load():
    stories, stories_masks, questions, questions_masks, answers, supports = batch
    break

In [9]:
print(stories.size(), stories_masks.size(), questions.size(), questions_masks.size(), answers.size())

torch.Size([32, 10, 6]) torch.Size([32, 10, 6]) torch.Size([32, 4]) torch.Size([32, 4]) torch.Size([32, 1])


Test

In [10]:
path_test = '../data/QA_bAbI_tasks/en-10k/qa1_single-supporting-fact_test.txt'

In [11]:
bAbI_test = bAbIDataset(path_test, train=False, vocab=bAbI.word2idx, return_masks=False)
test_loader = bAbIDataLoader(dataset=bAbI_test, batch_size=32, shuffle=True, to_tensor=True)

In [12]:
for batch in test_loader.load():
    stories, questions, answers, supports = batch
    break

In [13]:
print(len(stories), stories[0].size(), questions.size(), answers.size())

32 torch.Size([8, 6]) torch.Size([32, 4]) torch.Size([32, 1])


In [14]:
bAbI_test = bAbIDataset(path_test, train=False, vocab=bAbI.word2idx, return_masks=True)
test_loader = bAbIDataLoader(dataset=bAbI_test, batch_size=32, shuffle=True, to_tensor=True)

In [15]:
for batch in test_loader.load():
    stories, stories_masks, questions, questions_masks, answers, supports = batch
    break

In [16]:
print(len(stories), stories[0].size(), len(stories_masks), stories_masks[0].size(), questions.size(), answers.size())

32 torch.Size([2, 6]) 32 torch.Size([2, 6]) torch.Size([32, 4]) torch.Size([32, 1])
